# Postupci praćenja objekata u računalnom vidu radnim okvirom OpenCV
  
 ### Diplomski projekt: http://www.fer.unizg.hr/predmet/pro_dipl, siječanj 2018. FER, 2017/2018
 #### Autor: Tomislav Lokotar
 #### Mentor: izv. prof. dr. sc. Zoran Kalafatić

# 1. Uvod

OpenCV je vrlo raširen radni okvir koji nudi mnoge mogućnosti u kontekstu računalnog vida. Praćenje objekata u pokretu čest je problem kojeg je moguće upotrijebiti u razne svrhe. U ovom radu iznesen je kratak pregled implementiranih metoda za praćenje objekata, te njihova usporedba na javno dostupnim snimkama. Aktualna verzija u trenutku pisanja je OpenCV 3.4.0 u kojoj su implementirani slijedeći algoritmi praćenja: MedianFlow, MIL, GOTURN, KCF, Boosting, MOSSE i TLD. U sklopu projekta razvijen je i softver za usporedbu ugrađenih metoda praćenja. Skup podataka na kojem su algoritmi testirani je TB-50 [1]. TB-50 je skup 50 označenih sekvenci na kojima je označen najmanje jedan objekt koji se prati. Svi algoritmi, osim GOTURN s greškom u implementaciji [2], su uspješno testirani. Na slici 1 prikazan je prozor generiram skriptom za testiranje praćenja na videu s pokretnom kamerom, u procesu praćenja košarkaškog igrača algoritmom MIL.

![slika1](docs/kosarka.png)
<center>Slika 1: praćenje unaprijed označenog košarkaša na videosnimci</center> 


In [18]:
import cv2
import numpy as np
import cv_utils
import bench_utils
import tb_50

In [2]:
cv_utils.info()

Python version: 3.5.2
OpenCV version: 3.4.0

Available tracking algorithms:
TrackerTLD, TrackerKCF, TrackerGOTURN, TrackerMIL, TrackerMOSSE, TrackerMedianFlow, TrackerBoosting


In [7]:
algorithms = cv_utils.tracking_algorithms()

for alg in algorithms:
    print(alg)

TrackerTLD
TrackerKCF
TrackerGOTURN
TrackerMIL
TrackerMOSSE
TrackerMedianFlow
TrackerBoosting


U rječniku <i>algorithms</i> pohranjeni su uređeni parovi konstruktora objekata koji predstavljaju algoritme praćenja, te pripadajuće funkcije za njihovo instanciranje.

In [8]:
algorithms['TrackerGOTURN']

(cv2.TrackerGOTURN, <function TrackerGOTURN_create>)

In [11]:
goturn_create = algorithms['TrackerGOTURN'][1]

goturn_tracker = goturn_create()

U trenutnoj verziji OpenCV-a nije moguće pratiti objekt algoritmom GOTURN [3], pa je njegova funkcionalnost izuzeta iz evaluacije algoritama.

# 2 Analiza uspješnosti algoritama praćenja

## 2.1 Mjera uspješnosti praćenja

Kao mjera uspješnosti praćenja objekata korištena je funkcija $$C_{suc}(f)= \frac{\left | r\cap  r_g \right |}{\left | r \cup r_g\right |  }$$ na sličan način kao što su je koristili Janku, Koplik, Dulik[4]. Varijabla <i>r</i> predstavlja pravokutnik označen algoritmom praćenja, dok je <i>r<sub>g</sub></i> pravokutnik ručno označen na korištenim sekvencama. Idealna vrijednost, kad algoritam praćenja označi pravokutnik koji se poklapa s označenim, je 1. 0 je najmanja vrijednost i predstavlja promašaj u postupku praćenja. Mjerenje se vrši za svaku sličicu sekvence. Postupak testiranja počinje inicijalizacijom algoritma praćenja s ručno označenim pravokutnikom. Ukoliko algoritam praćenja u nekom trenutku postigne vrijednost funkcije <i>C<sub>suc</sub></i> ispod postavljene granice (obično 0.5 ili više), algoritam se reinicijalizira na ručno označeno područje, a broj reinicijalizacija se inkrementira. U situaciji praćenja objekta u stvarnom vremenu, reinicijalizacija se može poistovjetiti s upotrijebom detektora objekata.

## 2.2 Podaci za testiranje

Skup podataka na kojima su testirani algoritmi je TB-50[5]. TB-50 je javno dostupan označen skup podataka sa sekvencama u svrhu evalucije algoritama praćenja. Svaka sekvenca ima postavljenu jednu ili više zastavica ovisno o vrsti problema. Vrste problema uključuju nagle promjene osvjetljenja na objekt u praćenju, nagle promjene udaljenosti objekata, prekrivanje objekta, deformacije, zamućenost, brze pokrete, rotacije, napuštanje okvira snimke, tešku razlučivost objekta u odnosu na njegovu pozadinu, te snimke niske rezolucije.

Popis video datoteka:

In [22]:
benchmark_sequences = tb_50.get_all_videos_and_groundtruths()

for directory, video_path, ground_truth_file in benchmark_sequences:
    print(directory)

./TB-50/Basketball/
./TB-50/Biker/
./TB-50/Bird1/
./TB-50/BlurBody/
./TB-50/BlurCar2/
./TB-50/BlurFace/
./TB-50/BlurOwl/
./TB-50/Box/
./TB-50/Car1/
./TB-50/Car4/
./TB-50/CarDark/
./TB-50/CarScale/
./TB-50/ClifBar/
./TB-50/Couple/
./TB-50/Crowds/
./TB-50/Deer/
./TB-50/DragonBaby/
./TB-50/Dudek/
./TB-50/Girl/
./TB-50/Human6/
./TB-50/Human9/
./TB-50/Ironman/
./TB-50/Jump/
./TB-50/Jumping/
./TB-50/Matrix/
./TB-50/MotorRolling/
./TB-50/Panda/
./TB-50/RedTeam/
./TB-50/Shaking/
./TB-50/Singer2/
./TB-50/Skating1/
./TB-50/Skiing/
./TB-50/Soccer/
./TB-50/Surfer/
./TB-50/Sylvester/
./TB-50/Tiger2/
./TB-50/Trellis/
./TB-50/Walking/
./TB-50/Walking2/
./TB-50/Woman/


<!--bibtex

@Article{PER-GRA:2007,
  Author    = {P\'erez, Fernando and Granger, Brian E.},
  Title     = {{IP}ython: a System for Interactive Scientific Computing},
  Journal   = {Computing in Science and Engineering},
  Volume    = {9},
  Number    = {3},
  Pages     = {21--29},
  month     = may,
  year      = 2007,
  url       = "http://ipython.org",
  ISSN      = "1521-9615",
  doi       = {10.1109/MCSE.2007.53},
  publisher = {IEEE Computer Society},
}

@article{Papa2007,
  author = {Papa, David A. and Markov, Igor L.},
  journal = {Approximation algorithms and metaheuristics},
  pages = {1--38},
  title = {{Hypergraph partitioning and clustering}},
  url = {http://www.podload.org/pubs/book/part\_survey.pdf},
  year = {2007}
}

-->

Examples of citations: [CITE](#cite-PER-GRA:2007) or [CITE](#cite-Papa2007).